### Building Parameters

| Component | Width (X) | Depth (Y) | Floors | Height |
|-----------|-----------|-----------|--------|--------|
| **Podium** | 5 bays × 8m = 40m | 2 bays × 8m = 16m | Floors 0-12 (13 floors) | 81m |
| **Tower** | 3 bays × 8m = 24m | 2 bays × 8m = 16m | Floors 13-25 (13 floors) | 153m |

| Parameter | Value |
|-----------|-------|
| **Ground Floor Height** | 9.0m |
| **Typical Floor Height** | 6.0m |
| **Total Floors** | 26 (Floor 0 to Floor 25) |

---
## 1. Import Libraries and Setup

In [130]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

---
## 2. Building Geometry Parameters

In [131]:
BAY_WIDTH = 8.0

PODIUM_BAYS_X = 5
PODIUM_BAYS_Y = 2
PODIUM_FLOORS = 13

TOWER_BAYS_X = 3
TOWER_BAYS_Y = 2
TOTAL_FLOORS = 26

GROUND_FLOOR_HEIGHT = 9.0
TYPICAL_FLOOR_HEIGHT = 6.0

PODIUM_X_COORDS = np.arange(PODIUM_BAYS_X + 1) * BAY_WIDTH
TOWER_X_COORDS = np.array([8.0, 16.0, 24.0, 32.0])
Y_COORDS = np.arange(PODIUM_BAYS_Y + 1) * BAY_WIDTH

Z_COORDS = np.zeros(TOTAL_FLOORS)
Z_COORDS[0] = 0.0
Z_COORDS[1] = GROUND_FLOOR_HEIGHT
Z_COORDS[2:] = GROUND_FLOOR_HEIGHT + np.arange(1, TOTAL_FLOORS - 1) * TYPICAL_FLOOR_HEIGHT

print("PODIUM_X_COORDS:", PODIUM_X_COORDS)
print("TOWER_X_COORDS:", TOWER_X_COORDS)
print("Y_COORDS:", Y_COORDS)
print("Z_COORDS:", Z_COORDS)

PODIUM_X_COORDS: [ 0.  8. 16. 24. 32. 40.]
TOWER_X_COORDS: [ 8. 16. 24. 32.]
Y_COORDS: [ 0.  8. 16.]
Z_COORDS: [  0.   9.  15.  21.  27.  33.  39.  45.  51.  57.  63.  69.  75.  81.
  87.  93.  99. 105. 111. 117. 123. 129. 135. 141. 147. 153.]


---
## 3. Generate Position Matrix with Numpy

In [132]:
# Podium nodes: floors 0-12, full grid (6x3)
podium_floors = np.arange(PODIUM_FLOORS)
podium_xx, podium_yy, podium_ff = np.meshgrid(PODIUM_X_COORDS, Y_COORDS, podium_floors, indexing='ij')
podium_zz = Z_COORDS[podium_ff]

podium_x_flat = podium_xx.flatten()
podium_y_flat = podium_yy.flatten()
podium_z_flat = podium_zz.flatten()
podium_f_flat = podium_ff.flatten()

n_podium = len(podium_x_flat)
print(f"Podium nodes: {n_podium}")

# Tower nodes: floors 13-25, centered grid (4x3)
tower_floors = np.arange(PODIUM_FLOORS, TOTAL_FLOORS)
tower_xx, tower_yy, tower_ff = np.meshgrid(TOWER_X_COORDS, Y_COORDS, tower_floors, indexing='ij')
tower_zz = Z_COORDS[tower_ff]

tower_x_flat = tower_xx.flatten()
tower_y_flat = tower_yy.flatten()
tower_z_flat = tower_zz.flatten()
tower_f_flat = tower_ff.flatten()

n_tower = len(tower_x_flat)
print(f"Tower nodes: {n_tower}")

# Chevron Center Nodes: floors 0-25 (ALL FLOORS), X=20, Front/Back faces only (Y=0,16)
# User requested chevron braces passing through the middle (full height)
chev_x_coords = np.array([20.0])
chev_y_coords = np.array([Y_COORDS[0], Y_COORDS[-1]])
chev_floors = np.arange(TOTAL_FLOORS) # Changed from tower_floors to ALL floors

chev_xx, chev_yy, chev_ff = np.meshgrid(chev_x_coords, chev_y_coords, chev_floors, indexing='ij')
chev_zz = Z_COORDS[chev_ff]

chev_x_flat = chev_xx.flatten()
chev_y_flat = chev_yy.flatten()
chev_z_flat = chev_zz.flatten()
chev_f_flat = chev_ff.flatten()
n_chev = len(chev_x_flat)
print(f"Chevron nodes: {n_chev}")

# Combine all nodes
all_x = np.concatenate([podium_x_flat, tower_x_flat, chev_x_flat])
all_y = np.concatenate([podium_y_flat, tower_y_flat, chev_y_flat])
all_z = np.concatenate([podium_z_flat, tower_z_flat, chev_z_flat])
all_floor = np.concatenate([podium_f_flat, tower_f_flat, chev_f_flat])
all_zone = np.array(['podium'] * n_podium + ['tower'] * n_tower + ['chevron_node'] * n_chev)

n_nodes = len(all_x)
node_ids = np.arange(n_nodes)

position_df = pd.DataFrame({
    'node_id': node_ids,
    'x': all_x,
    'y': all_y,
    'z': all_z,
    'floor': all_floor.astype(int),
    'zone': all_zone
})

print(f"\nTotal nodes: {n_nodes}")
print(f"Position Matrix Shape: {position_df.shape}")
display(position_df.head(20))


Podium nodes: 234
Tower nodes: 156
Chevron nodes: 52

Total nodes: 442
Position Matrix Shape: (442, 6)


,node_id,x,y,z,floor,zone
0,0,0.0,0.0,0.0,0,podium
1,1,0.0,0.0,9.0,1,podium
2,2,0.0,0.0,15.0,2,podium
3,3,0.0,0.0,21.0,3,podium
4,4,0.0,0.0,27.0,4,podium
5,5,0.0,0.0,33.0,5,podium
6,6,0.0,0.0,39.0,6,podium
7,7,0.0,0.0,45.0,7,podium
8,8,0.0,0.0,51.0,8,podium
9,9,0.0,0.0,57.0,9,podium


In [133]:
# Create node lookup: (floor, x, y) -> node_id
node_lookup = {}
for i in range(n_nodes):
    key = (int(all_floor[i]), float(all_x[i]), float(all_y[i]))
    node_lookup[key] = i

print("Node lookup examples:")
print(f"  Floor 0, (0, 0): Node {node_lookup.get((0, 0.0, 0.0), 'N/A')}")
print(f"  Floor 12, (0, 0): Node {node_lookup.get((12, 0.0, 0.0), 'N/A')} (last podium)")
print(f"  Floor 13, (8, 0): Node {node_lookup.get((13, 8.0, 0.0), 'N/A')} (first tower)")
print(f"  Floor 13, (0, 0): Node {node_lookup.get((13, 0.0, 0.0), 'N/A')} (edge - should be N/A)")

Node lookup examples:
  Floor 0, (0, 0): Node 0
  Floor 12, (0, 0): Node 12 (last podium)
  Floor 13, (8, 0): Node 234 (first tower)
  Floor 13, (0, 0): Node N/A (edge - should be N/A)


---
## 4. Generate Connectivity with Numpy Arrays

**Bracing Strategy:**
- Front/Back faces (Y=0, Y=16): X-brace on ALL bays, ALL floors
- Left/Right faces: X-brace only on CORNER bays (reduced)
- Tower center: Chevron braces (inverted V)

In [134]:
# Helper arrays for element creation
elements_list = []

# ============================================
# BEAMS X-DIRECTION (numpy vectorized)
# ============================================
beam_x_nodes = []

# Podium beams X
for f in range(PODIUM_FLOORS):
    for y in Y_COORDS:
        for i in range(len(PODIUM_X_COORDS) - 1):
            n1 = node_lookup.get((f, PODIUM_X_COORDS[i], y))
            n2 = node_lookup.get((f, PODIUM_X_COORDS[i+1], y))
            if n1 is not None and n2 is not None:
                beam_x_nodes.append([n1, n2])

# Tower beams X
for f in range(PODIUM_FLOORS, TOTAL_FLOORS):
    for y in Y_COORDS:
        for i in range(len(TOWER_X_COORDS) - 1):
            n1 = node_lookup.get((f, TOWER_X_COORDS[i], y))
            n2 = node_lookup.get((f, TOWER_X_COORDS[i+1], y))
            if n1 is not None and n2 is not None:
                beam_x_nodes.append([n1, n2])

beam_x_arr = np.array(beam_x_nodes)
print(f"Beam X elements: {len(beam_x_arr)}")

# ============================================
# BEAMS Y-DIRECTION
# ============================================
beam_y_nodes = []

# Podium beams Y
for f in range(PODIUM_FLOORS):
    for x in PODIUM_X_COORDS:
        for i in range(len(Y_COORDS) - 1):
            n1 = node_lookup.get((f, x, Y_COORDS[i]))
            n2 = node_lookup.get((f, x, Y_COORDS[i+1]))
            if n1 is not None and n2 is not None:
                beam_y_nodes.append([n1, n2])

# Tower beams Y
for f in range(PODIUM_FLOORS, TOTAL_FLOORS):
    for x in TOWER_X_COORDS:
        for i in range(len(Y_COORDS) - 1):
            n1 = node_lookup.get((f, x, Y_COORDS[i]))
            n2 = node_lookup.get((f, x, Y_COORDS[i+1]))
            if n1 is not None and n2 is not None:
                beam_y_nodes.append([n1, n2])

beam_y_arr = np.array(beam_y_nodes)
print(f"Beam Y elements: {len(beam_y_arr)}")

# ============================================
# COLUMNS (vertical)
# ============================================
column_nodes = []

# Podium columns (floors 0-11 to 1-12)
for f in range(PODIUM_FLOORS - 1):
    for x in PODIUM_X_COORDS:
        for y in Y_COORDS:
            n1 = node_lookup.get((f, x, y))
            n2 = node_lookup.get((f+1, x, y))
            if n1 is not None and n2 is not None:
                column_nodes.append([n1, n2])

# Transition columns (floor 12 to 13) - only tower footprint
for x in TOWER_X_COORDS:
    for y in Y_COORDS:
        n1 = node_lookup.get((PODIUM_FLOORS - 1, x, y))
        n2 = node_lookup.get((PODIUM_FLOORS, x, y))
        if n1 is not None and n2 is not None:
            column_nodes.append([n1, n2])

# Tower columns (floors 13-24 to 14-25)
for f in range(PODIUM_FLOORS, TOTAL_FLOORS - 1):
    for x in TOWER_X_COORDS:
        for y in Y_COORDS:
            n1 = node_lookup.get((f, x, y))
            n2 = node_lookup.get((f+1, x, y))
            if n1 is not None and n2 is not None:
                column_nodes.append([n1, n2])

column_arr = np.array(column_nodes)
print(f"Column elements: {len(column_arr)}")

Beam X elements: 312
Beam Y elements: 260
Column elements: 372


In [135]:
# ============================================
# BRACES XZ - FRONT/BACK FACES + INTERNAL
# MEGA BRACES: Spanning 2 Floors (0->2, 2->4...)
# Skip Center Bay on Faces (for Chevron).
# ============================================
brace_xz_nodes = []

# Iterate floors with step 2
for f in range(0, TOTAL_FLOORS - 1, 2):
    f_top = f + 2
    if f_top >= TOTAL_FLOORS:
        break # Cannot do 2-story brace at very top if odd count remaining
        
    for y in Y_COORDS:
        # Iterate all potential bays (Podium width)
        # We check checks for node existence to handle Tower/Podium width differences automatically.
        all_x_candidates = [0.0, 8.0, 16.0, 24.0, 32.0, 40.0]
        
        for i in range(len(all_x_candidates) - 1):
            x1, x2 = all_x_candidates[i], all_x_candidates[i+1]
            
            # Check Face vs Internal
            is_face = (y == Y_COORDS[0] or y == Y_COORDS[-1])
            
            # Center Bay logic (16-24)
            is_center_bay = (x1 == 16.0 and x2 == 24.0)
            
            # Skip Center Bay entirely (Core wall + Chevron area)
            if is_center_bay:
                continue
            
            # Check corner existence for 2-story bay
            n_bl = node_lookup.get((f, x1, y))
            n_br = node_lookup.get((f, x2, y))
            n_tl = node_lookup.get((f_top, x1, y))
            n_tr = node_lookup.get((f_top, x2, y))
            
            if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                brace_xz_nodes.append([n_bl, n_tr])
                brace_xz_nodes.append([n_br, n_tl])

brace_xz_arr = np.array(brace_xz_nodes)
print(f"Brace XZ (2-story span) elements: {len(brace_xz_arr)}")


Brace XZ (2-story span) elements: 216


In [136]:
# ============================================
# BRACES YZ - LEFT/RIGHT FACES (edge X only)
# MEGA BRACES: Spanning 2 Floors (0->2, 2->4...)
# ============================================
brace_yz_nodes = []

# Iterate floors with step 2
for f in range(0, TOTAL_FLOORS - 1, 2):
    f_top = f + 2
    if f_top >= TOTAL_FLOORS:
        break
    
    # X positions for left/right edges depend on floor zone
    # Podium: Left=0, Right=40
    # Tower: Left=8, Right=32
    # We check for node existence to handle boundary automatically.
    
    # Left edge candidates
    left_x_candidates = [0.0, 8.0]  # Podium=0, Tower=8
    # Right edge candidates
    right_x_candidates = [40.0, 32.0]  # Podium=40, Tower=32
    
    for x_list in [left_x_candidates, right_x_candidates]:
        for x in x_list:
            # Iterate Y bays
            for j in range(len(Y_COORDS) - 1):
                y1, y2 = Y_COORDS[j], Y_COORDS[j+1]
                
                n_bl = node_lookup.get((f, x, y1))
                n_br = node_lookup.get((f, x, y2))
                n_tl = node_lookup.get((f_top, x, y1))
                n_tr = node_lookup.get((f_top, x, y2))
                
                if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                    brace_yz_nodes.append([n_bl, n_tr])
                    brace_yz_nodes.append([n_br, n_tl])

brace_yz_arr = np.array(brace_yz_nodes) if brace_yz_nodes else np.array([]).reshape(0, 2)
print(f"Brace YZ (2-story span, left/right) elements: {len(brace_yz_arr)}")
print(f"\nBrace ratio (XZ:YZ) = {len(brace_xz_arr)}:{len(brace_yz_arr)} = {len(brace_xz_arr)/max(len(brace_yz_arr),1):.1f}:1")


Brace YZ (2-story span, left/right) elements: 144

Brace ratio (XZ:YZ) = 216:144 = 1.5:1


In [137]:
# ============================================
# CHEVRON BRACES - Center Bay (Inverted V)
# Floors 0 to Top. Front/Back faces only.
# ============================================
chevron_nodes = []

center_x_left = 16.0
center_x_right = 24.0
center_x_mid = 20.0

# Floors 0 to Top
for f in range(TOTAL_FLOORS - 1):
    for y in [Y_COORDS[0], Y_COORDS[-1]]: # Faces only
        # Inverted V: Base at floor f, Apex at floor f+1
        n_bl = node_lookup.get((f, center_x_left, y))
        n_br = node_lookup.get((f, center_x_right, y))
        n_apex = node_lookup.get((f+1, center_x_mid, y))
        
        if all(n is not None for n in [n_bl, n_br, n_apex]):
            chevron_nodes.append([n_bl, n_apex])
            chevron_nodes.append([n_br, n_apex])

chevron_arr = np.array(chevron_nodes) if chevron_nodes else np.array([]).reshape(0, 2)
print(f"Chevron brace elements: {len(chevron_arr)}")


Chevron brace elements: 100


In [138]:
# ============================================
# SPACE BRACES - Podium to Tower Transition
# 3D diagonal braces transferring loads from
# wider podium (X=0,40) to narrower tower (X=8,32)
# ============================================
space_brace_nodes = []

# Transition floor: Podium top (floor 12) to Tower bottom (floor 13)
f_podium_top = PODIUM_FLOORS - 1  # 12
f_tower_bottom = PODIUM_FLOORS    # 13

# Podium outer X coords: 0, 40
# Tower inner X coords: 8, 32
# Connect each outer podium corner to corresponding inner tower corner

# Left side: X=0 (podium) -> X=8 (tower)
# Right side: X=40 (podium) -> X=32 (tower)

transitions = [
    (0.0, 8.0),   # Left outer to left inner
    (40.0, 32.0), # Right outer to right inner
]

for x_pod, x_tow in transitions:
    for y in Y_COORDS:
        # Podium top node
        n_pod = node_lookup.get((f_podium_top, x_pod, y))
        
        # Connect to ALL tower bottom nodes on that Y-line
        for y_tow in Y_COORDS:
            n_tow = node_lookup.get((f_tower_bottom, x_tow, y_tow))
            if n_pod is not None and n_tow is not None:
                space_brace_nodes.append([n_pod, n_tow])

# Also connect diagonally across (podium X=0 to tower X=8 across Y)
# This creates the "web" pattern shown in user's image

# Cross connections: left podium to all tower left, right podium to all tower right
for x_pod, x_tow in transitions:
    for y_pod in Y_COORDS:
        n_pod = node_lookup.get((f_podium_top, x_pod, y_pod))
        # Connect to tower nodes at same X but different Ys
        for y_tow in Y_COORDS:
            if y_pod != y_tow:  # Cross connection
                n_tow = node_lookup.get((f_tower_bottom, x_tow, y_tow))
                # Already added above, skip duplicates
                pass

space_brace_arr = np.array(space_brace_nodes) if space_brace_nodes else np.array([]).reshape(0, 2)
print(f"Space brace elements (transition): {len(space_brace_arr)}")


Space brace elements (transition): 18


In [139]:
# ============================================
# CENTRAL CORE - Shear Walls (Perde Duvarlar)
# 4-sided core box running full height
# Using balsa panels (3mm thick)
# Core location: X=16-24, Y=0-16 (center bay)
# ============================================
core_wall_nodes = []

# Core boundary coordinates
core_x_left = 16.0
core_x_right = 24.0
core_y_front = 0.0
core_y_back = 16.0
core_y_mid = 8.0  # Internal Y grid line

# Shear walls are vertical panels connecting floor nodes
# We'll create diagonal bracing pattern on each wall face
# This simulates the panel behavior in the model

# FRONT WALL (Y=0): X from 16-24
# BACK WALL (Y=16): X from 16-24
# LEFT WALL (X=16): Y from 0-16
# RIGHT WALL (X=24): Y from 0-16

for f in range(TOTAL_FLOORS - 1):
    f_top = f + 1
    
    # FRONT WALL (Y=0) - X bay 16-24
    n_bl = node_lookup.get((f, core_x_left, core_y_front))
    n_br = node_lookup.get((f, core_x_right, core_y_front))
    n_tl = node_lookup.get((f_top, core_x_left, core_y_front))
    n_tr = node_lookup.get((f_top, core_x_right, core_y_front))
    if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
        core_wall_nodes.append([n_bl, n_tr])  # Diagonal 1
        core_wall_nodes.append([n_br, n_tl])  # Diagonal 2
    
    # BACK WALL (Y=16) - X bay 16-24  
    n_bl = node_lookup.get((f, core_x_left, core_y_back))
    n_br = node_lookup.get((f, core_x_right, core_y_back))
    n_tl = node_lookup.get((f_top, core_x_left, core_y_back))
    n_tr = node_lookup.get((f_top, core_x_right, core_y_back))
    if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
        core_wall_nodes.append([n_bl, n_tr])
        core_wall_nodes.append([n_br, n_tl])
    
    # LEFT WALL (X=16) - Y bay 0-8
    n_bl = node_lookup.get((f, core_x_left, core_y_front))
    n_br = node_lookup.get((f, core_x_left, core_y_mid))
    n_tl = node_lookup.get((f_top, core_x_left, core_y_front))
    n_tr = node_lookup.get((f_top, core_x_left, core_y_mid))
    if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
        core_wall_nodes.append([n_bl, n_tr])
        core_wall_nodes.append([n_br, n_tl])
    
    # LEFT WALL (X=16) - Y bay 8-16
    n_bl = node_lookup.get((f, core_x_left, core_y_mid))
    n_br = node_lookup.get((f, core_x_left, core_y_back))
    n_tl = node_lookup.get((f_top, core_x_left, core_y_mid))
    n_tr = node_lookup.get((f_top, core_x_left, core_y_back))
    if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
        core_wall_nodes.append([n_bl, n_tr])
        core_wall_nodes.append([n_br, n_tl])
    
    # RIGHT WALL (X=24) - Y bay 0-8
    n_bl = node_lookup.get((f, core_x_right, core_y_front))
    n_br = node_lookup.get((f, core_x_right, core_y_mid))
    n_tl = node_lookup.get((f_top, core_x_right, core_y_front))
    n_tr = node_lookup.get((f_top, core_x_right, core_y_mid))
    if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
        core_wall_nodes.append([n_bl, n_tr])
        core_wall_nodes.append([n_br, n_tl])
    
    # RIGHT WALL (X=24) - Y bay 8-16
    n_bl = node_lookup.get((f, core_x_right, core_y_mid))
    n_br = node_lookup.get((f, core_x_right, core_y_back))
    n_tl = node_lookup.get((f_top, core_x_right, core_y_mid))
    n_tr = node_lookup.get((f_top, core_x_right, core_y_back))
    if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
        core_wall_nodes.append([n_bl, n_tr])
        core_wall_nodes.append([n_br, n_tl])

core_wall_arr = np.array(core_wall_nodes) if core_wall_nodes else np.array([]).reshape(0, 2)
print(f"Core wall elements: {len(core_wall_arr)}")


Core wall elements: 300


In [140]:
# ============================================
# FLOOR BRACES (slab diagonals)
# ============================================
brace_floor_nodes = []

# Podium floor braces
for f in range(PODIUM_FLOORS):
    for i in range(len(PODIUM_X_COORDS) - 1):
        for j in range(len(Y_COORDS) - 1):
            x1, x2 = PODIUM_X_COORDS[i], PODIUM_X_COORDS[i+1]
            y1, y2 = Y_COORDS[j], Y_COORDS[j+1]
            n_bl = node_lookup.get((f, x1, y1))
            n_br = node_lookup.get((f, x2, y1))
            n_tl = node_lookup.get((f, x1, y2))
            n_tr = node_lookup.get((f, x2, y2))
            if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                brace_floor_nodes.append([n_bl, n_tr])
                brace_floor_nodes.append([n_br, n_tl])

# Tower floor braces
for f in range(PODIUM_FLOORS, TOTAL_FLOORS):
    for i in range(len(TOWER_X_COORDS) - 1):
        for j in range(len(Y_COORDS) - 1):
            x1, x2 = TOWER_X_COORDS[i], TOWER_X_COORDS[i+1]
            y1, y2 = Y_COORDS[j], Y_COORDS[j+1]
            n_bl = node_lookup.get((f, x1, y1))
            n_br = node_lookup.get((f, x2, y1))
            n_tl = node_lookup.get((f, x1, y2))
            n_tr = node_lookup.get((f, x2, y2))
            if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                brace_floor_nodes.append([n_bl, n_tr])
                brace_floor_nodes.append([n_br, n_tl])

brace_floor_arr = np.array(brace_floor_nodes)
print(f"Floor brace elements: {len(brace_floor_arr)}")

Floor brace elements: 416


In [141]:
# ============================================
# Combine all elements into connectivity DataFrame
# ============================================

# Calculate lengths using numpy
coords = position_df[['x', 'y', 'z']].values

all_elements = []
element_id = 0

element_types = [
    ('beam_x', beam_x_arr),
    ('beam_y', beam_y_arr),
    ('column', column_arr),
    ('brace_xz', brace_xz_arr),
    ('brace_yz', brace_yz_arr),
    ('chevron', chevron_arr),
    ('brace_space', space_brace_arr),
    ('core_wall', core_wall_arr),
    ('brace_floor', brace_floor_arr)
]

for etype, arr in element_types:
    if len(arr) == 0:
        continue
    for row in arr:
        n1, n2 = int(row[0]), int(row[1])
        c1, c2 = coords[n1], coords[n2]
        length = np.sqrt(np.sum((c2 - c1) ** 2))
        all_elements.append({
            'element_id': element_id,
            'node_i': n1,
            'node_j': n2,
            'element_type': etype,
            'length': round(length, 4)
        })
        element_id += 1

connectivity_df = pd.DataFrame(all_elements)

print("=" * 70)
print("CONNECTIVITY MATRIX")
print("=" * 70)
print(f"\nTotal elements: {len(connectivity_df)}")
print(f"\nElements by type:")
print(connectivity_df['element_type'].value_counts())

n_xz = len(connectivity_df[connectivity_df['element_type'] == 'brace_xz'])
n_yz = len(connectivity_df[connectivity_df['element_type'] == 'brace_yz'])
n_chevron = len(connectivity_df[connectivity_df['element_type'] == 'chevron'])
print(f"\nBrace ratio (XZ : YZ) = {n_xz} : {n_yz} = {n_xz/max(n_yz,1):.1f} : 1")
print(f"Chevron braces: {n_chevron}")

CONNECTIVITY MATRIX

Total elements: 2138

Elements by type:
element_type
brace_floor    416
column         372
beam_x         312
core_wall      300
beam_y         260
brace_xz       216
brace_yz       144
chevron        100
brace_space     18
Name: count, dtype: int64

Brace ratio (XZ : YZ) = 216 : 144 = 1.5 : 1
Chevron braces: 100


In [142]:
# Create adjacency matrix
adjacency_matrix = np.zeros((n_nodes, n_nodes), dtype=int)

for _, row in connectivity_df.iterrows():
    i, j = int(row['node_i']), int(row['node_j'])
    adjacency_matrix[i, j] = 1
    adjacency_matrix[j, i] = 1

print(f"Adjacency Matrix Shape: {adjacency_matrix.shape}")
print(f"Total connections (edges): {np.sum(adjacency_matrix) // 2}")

Adjacency Matrix Shape: (442, 442)
Total connections (edges): 2138


---
## 5. Interactive 3D Visualization

In [143]:
# 3D Building View
fig_3d = go.Figure()

element_styles = {
    'column':       {'color': 'blue',      'width': 3},
    'beam_x':       {'color': 'green',     'width': 2},
    'beam_y':       {'color': 'orange',    'width': 2},
    'brace_xz':     {'color': 'red',       'width': 2},
    'brace_yz':     {'color': 'purple',    'width': 2},
    'chevron':      {'color': 'cyan',      'width': 2.5},
    'brace_floor':  {'color': 'lightgray', 'width': 1},
    'brace_space':  {'color': 'black',     'width': 4},
    'core_wall':    {'color': 'brown',     'width': 3}
}

for etype, style in element_styles.items():
    elements = connectivity_df[connectivity_df['element_type'] == etype]
    if len(elements) == 0:
        continue
    
    x_lines, y_lines, z_lines = [], [], []
    for _, row in elements.iterrows():
        n1_coords = coords[int(row['node_i'])]
        n2_coords = coords[int(row['node_j'])]
        x_lines.extend([n1_coords[0], n2_coords[0], None])
        y_lines.extend([n1_coords[1], n2_coords[1], None])
        z_lines.extend([n1_coords[2], n2_coords[2], None])
    
    fig_3d.add_trace(go.Scatter3d(
        x=x_lines, y=y_lines, z=z_lines,
        mode='lines',
        name=etype.replace('_', ' ').title(),
        line=dict(color=style['color'], width=style['width']),
        hoverinfo='name'
    ))

fig_3d.add_trace(go.Scatter3d(
    x=all_x, y=all_y, z=all_z,
    mode='markers',
    name='Nodes',
    marker=dict(size=3, color=all_floor, colorscale='Viridis', opacity=0.8),
    hovertemplate='Node %{customdata}<br>X: %{x}m<br>Y: %{y}m<br>Z: %{z}m<extra></extra>',
    customdata=node_ids
))

fig_3d.update_layout(
    title='Stepped Building - 3D View',
    scene=dict(
        xaxis_title='X [m]',
        yaxis_title='Y [m]',
        zaxis_title='Z [m]',
        aspectmode='data',
        camera=dict(eye=dict(x=1.5, y=1.5, z=0.7))
    ),
    width=1000, height=800
)
fig_3d.show()

---
## 6. Floor Plan Views

In [144]:
# Floor 0 Plan - Shows ALL element types
floor_num = 0
floor_mask = position_df['floor'] == floor_num
floor_nodes = position_df[floor_mask]
floor_node_ids = set(floor_nodes['node_id'])

# Get ALL elements on this floor (not just beams)
floor_elements = connectivity_df[
    (connectivity_df['node_i'].isin(floor_node_ids)) & 
    (connectivity_df['node_j'].isin(floor_node_ids))
]

fig_floor0 = go.Figure()

# Style for each element type in plan view
floor_styles = {
    'beam_x':       {'color': 'green',     'width': 4},
    'beam_y':       {'color': 'orange',    'width': 4},
    'brace_floor':  {'color': 'lightgray', 'width': 2},
    'core_wall':    {'color': 'brown',     'width': 5},
    'chevron':      {'color': 'cyan',      'width': 3},
    'brace_xz':     {'color': 'red',       'width': 2},
    'brace_yz':     {'color': 'purple',    'width': 2},
}

for etype, style in floor_styles.items():
    elements = floor_elements[floor_elements['element_type'] == etype]
    if len(elements) == 0:
        continue
    
    x_lines, y_lines = [], []
    for _, row in elements.iterrows():
        c1, c2 = coords[int(row['node_i'])], coords[int(row['node_j'])]
        x_lines.extend([c1[0], c2[0], None])
        y_lines.extend([c1[1], c2[1], None])
    
    fig_floor0.add_trace(go.Scatter(
        x=x_lines, y=y_lines,
        mode='lines', 
        line=dict(color=style['color'], width=style['width']),
        name=etype.replace('_', ' ').title(),
        hoverinfo='name'
    ))

# Add column markers
fig_floor0.add_trace(go.Scatter(
    x=floor_nodes['x'], y=floor_nodes['y'],
    mode='markers+text',
    marker=dict(size=12, color='blue', symbol='square'),
    text=[f"({int(r['x'])},{int(r['y'])})" for _, r in floor_nodes.iterrows()],
    textposition='top center', 
    textfont=dict(size=8),
    name='Columns'
))

# Add core wall boundary rectangle
fig_floor0.add_shape(
    type="rect", x0=16, y0=0, x1=24, y1=16,
    line=dict(color="brown", width=2, dash="dash"),
    fillcolor="rgba(165, 42, 42, 0.1)"
)
fig_floor0.add_annotation(x=20, y=8, text="CORE", showarrow=False, font=dict(size=14, color="brown"))

fig_floor0.update_layout(
    title=f'Floor {floor_num} Plan (Podium) | Z = {Z_COORDS[floor_num]}m | Elements: {len(floor_elements)}',
    xaxis_title='X [m]', yaxis_title='Y [m]',
    xaxis=dict(scaleanchor='y', range=[-5, 45]),
    yaxis=dict(range=[-5, 21]),
    width=1000, height=600,
    legend=dict(x=1.02, y=1)
)
fig_floor0.show()

In [145]:
# Floor 15 Plan (Tower) - Shows ALL element types
floor_num = 15
floor_mask = position_df['floor'] == floor_num
floor_nodes = position_df[floor_mask]
floor_node_ids = set(floor_nodes['node_id'])

# Get ALL elements on this floor
floor_elements = connectivity_df[
    (connectivity_df['node_i'].isin(floor_node_ids)) & 
    (connectivity_df['node_j'].isin(floor_node_ids))
]

fig_floor15 = go.Figure()

# Style for each element type
floor_styles = {
    'beam_x':       {'color': 'green',     'width': 4},
    'beam_y':       {'color': 'orange',    'width': 4},
    'brace_floor':  {'color': 'lightgray', 'width': 2},
    'core_wall':    {'color': 'brown',     'width': 5},
    'chevron':      {'color': 'cyan',      'width': 3},
    'brace_xz':     {'color': 'red',       'width': 2},
    'brace_yz':     {'color': 'purple',    'width': 2},
}

for etype, style in floor_styles.items():
    elements = floor_elements[floor_elements['element_type'] == etype]
    if len(elements) == 0:
        continue
    
    x_lines, y_lines = [], []
    for _, row in elements.iterrows():
        c1, c2 = coords[int(row['node_i'])], coords[int(row['node_j'])]
        x_lines.extend([c1[0], c2[0], None])
        y_lines.extend([c1[1], c2[1], None])
    
    fig_floor15.add_trace(go.Scatter(
        x=x_lines, y=y_lines,
        mode='lines', 
        line=dict(color=style['color'], width=style['width']),
        name=etype.replace('_', ' ').title(),
        hoverinfo='name'
    ))

# Add column markers
fig_floor15.add_trace(go.Scatter(
    x=floor_nodes['x'], y=floor_nodes['y'],
    mode='markers+text',
    marker=dict(size=12, color='blue', symbol='square'),
    text=[f"({int(r['x'])},{int(r['y'])})" for _, r in floor_nodes.iterrows()],
    textposition='top center',
    textfont=dict(size=8),
    name='Columns'
))

# Add core wall boundary
fig_floor15.add_shape(
    type="rect", x0=16, y0=0, x1=24, y1=16,
    line=dict(color="brown", width=2, dash="dash"),
    fillcolor="rgba(165, 42, 42, 0.1)"
)
fig_floor15.add_annotation(x=20, y=8, text="CORE", showarrow=False, font=dict(size=14, color="brown"))

# Add tower boundary
fig_floor15.add_shape(
    type="rect", x0=8, y0=0, x1=32, y1=16,
    line=dict(color="blue", width=1, dash="dot"),
)

fig_floor15.update_layout(
    title=f'Floor {floor_num} Plan (Tower) | Z = {Z_COORDS[floor_num]}m | Elements: {len(floor_elements)}',
    xaxis_title='X [m]', yaxis_title='Y [m]',
    xaxis=dict(scaleanchor='y', range=[-5, 45]),
    yaxis=dict(range=[-5, 21]),
    width=1000, height=600,
    legend=dict(x=1.02, y=1)
)
fig_floor15.show()

---
## 7. Front Elevation View (X-Z Plane at Y=0)

In [146]:
# Front Elevation - Y=0 face (shows ALL elements on this face)
front_mask = position_df['y'] == 0
front_nodes = position_df[front_mask]
front_node_ids = set(front_nodes['node_id'])

# Also include chevron nodes (at Y=0 or Y=16)
chevron_front_mask = (position_df['y'] == 0) | (position_df['zone'] == 'chevron_node')
chevron_front_nodes = position_df[chevron_front_mask & (position_df['y'] == 0)]
all_front_ids = set(front_nodes['node_id']) | set(position_df[(position_df['zone'] == 'chevron_node') & (position_df['y'] == 0)]['node_id'])

front_elements = connectivity_df[
    (connectivity_df['node_i'].isin(all_front_ids)) & 
    (connectivity_df['node_j'].isin(all_front_ids))
]

fig_front = go.Figure()

# Styles for front elevation
styles_front = {
    'column':     {'color': 'blue',   'width': 3},
    'beam_x':     {'color': 'green',  'width': 2},
    'brace_xz':   {'color': 'red',    'width': 2},
    'chevron':    {'color': 'cyan',   'width': 3},
    'core_wall':  {'color': 'brown',  'width': 4},
    'brace_space': {'color': 'black', 'width': 3},
}

for etype, style in styles_front.items():
    elements = front_elements[front_elements['element_type'] == etype]
    if len(elements) == 0:
        continue
    
    x_lines, z_lines = [], []
    for _, row in elements.iterrows():
        c1, c2 = coords[int(row['node_i'])], coords[int(row['node_j'])]
        x_lines.extend([c1[0], c2[0], None])
        z_lines.extend([c1[2], c2[2], None])
    
    fig_front.add_trace(go.Scatter(
        x=x_lines, y=z_lines,
        mode='lines', 
        line=dict(color=style['color'], width=style['width']),
        name=etype.replace('_', ' ').title(),
        hoverinfo='name'
    ))

# Add node markers
fig_front.add_trace(go.Scatter(
    x=front_nodes['x'], y=front_nodes['z'],
    mode='markers', 
    marker=dict(size=4, color='darkgreen'),
    name='Nodes'
))

# Annotations
podium_top = Z_COORDS[PODIUM_FLOORS - 1]
total_height = Z_COORDS[-1]

# Podium/Tower boundaries
fig_front.add_shape(type="rect", x0=0, y0=0, x1=40, y1=podium_top,
                    line=dict(color="gray", width=1, dash="dot"), fillcolor="rgba(0,0,255,0.02)")
fig_front.add_shape(type="rect", x0=8, y0=podium_top, x1=32, y1=total_height,
                    line=dict(color="gray", width=1, dash="dot"), fillcolor="rgba(0,255,0,0.02)")

# Core boundary
fig_front.add_shape(type="rect", x0=16, y0=0, x1=24, y1=total_height,
                    line=dict(color="brown", width=2, dash="dash"), fillcolor="rgba(165,42,42,0.05)")

fig_front.add_annotation(x=20, y=-8, text="40m (Podium)", showarrow=False, font=dict(size=12))
fig_front.add_annotation(x=20, y=total_height+8, text="24m (Tower)", showarrow=False, font=dict(size=12))
fig_front.add_annotation(x=20, y=total_height/2, text="CORE", showarrow=False, font=dict(size=10, color="brown"))
fig_front.add_hline(y=podium_top, line_dash="dash", line_color="gray", annotation_text="Podium Top")

fig_front.update_layout(
    title=f'Front Elevation (Y=0) | Height: {total_height:.0f}m | Elements: {len(front_elements)}',
    xaxis_title='X [m]', yaxis_title='Z [m]',
    xaxis=dict(range=[-10, 50]),
    yaxis=dict(range=[-15, total_height+20], scaleanchor='x'),
    width=800, height=1000,
    legend=dict(x=1.02, y=1)
)
fig_front.show()

---
## 8. Side Elevation View (Y-Z Plane)

In [147]:
# Side Elevations (Left and Right) - Shows ALL elements
from plotly.subplots import make_subplots

fig_sides = make_subplots(rows=1, cols=2, subplot_titles=['Left Elevation (X=0/8)', 'Right Elevation (X=40/32)'],
                          horizontal_spacing=0.1)

# Styles for side elevation
styles_side = {
    'column':      {'color': 'blue',   'width': 4},
    'beam_y':      {'color': 'green',  'width': 3},
    'brace_yz':    {'color': 'red',    'width': 2},
    'core_wall':   {'color': 'brown',  'width': 4},
    'brace_space': {'color': 'black',  'width': 3},
}

# Left side: Podium X=0, Tower X=8
# Right side: Podium X=40, Tower X=32
sides = [
    {'name': 'Left',  'x_coords': [0.0, 8.0],  'col': 1},
    {'name': 'Right', 'x_coords': [40.0, 32.0], 'col': 2}
]

for side in sides:
    # Get all nodes on this side face
    side_mask = position_df['x'].isin(side['x_coords'])
    side_nodes = position_df[side_mask]
    side_node_ids = set(side_nodes['node_id'])
    
    # Get elements
    side_elements = connectivity_df[
        (connectivity_df['node_i'].isin(side_node_ids)) & 
        (connectivity_df['node_j'].isin(side_node_ids))
    ]
    
    for etype, style in styles_side.items():
        elements = side_elements[side_elements['element_type'] == etype]
        if len(elements) == 0:
            continue
        
        y_lines, z_lines = [], []
        for _, row in elements.iterrows():
            c1 = coords[int(row['node_i'])]
            c2 = coords[int(row['node_j'])]
            y_lines.extend([c1[1], c2[1], None])
            z_lines.extend([c1[2], c2[2], None])
        
        fig_sides.add_trace(go.Scatter(
            x=y_lines, y=z_lines,
            mode='lines', 
            line=dict(color=style['color'], width=style['width']),
            name=etype.replace('_', ' ').title() if side['col'] == 1 else None,
            showlegend=(side['col'] == 1),
            hoverinfo='name'
        ), row=1, col=side['col'])

    # Add node markers
    fig_sides.add_trace(go.Scatter(
        x=side_nodes['y'], y=side_nodes['z'],
        mode='markers', 
        marker=dict(size=4, color='darkgreen'),
        showlegend=False
    ), row=1, col=side['col'])

    # Add reference lines
    fig_sides.add_hline(y=podium_top, line_dash="dash", line_color="gray", row=1, col=side['col'])

fig_sides.update_xaxes(title_text='Y [m]', range=[-2, 18])
fig_sides.update_yaxes(title_text='Z [m]', range=[-10, total_height+10])

fig_sides.update_layout(
    title='Side Elevations (Left & Right Faces)',
    width=1000, height=900,
    legend=dict(x=1.02, y=1)
)
fig_sides.show()

---
## 9. Statistics Summary

In [148]:
print("=" * 70)
print("BUILDING STRUCTURAL SUMMARY")
print("=" * 70)

print(f"\nGEOMETRY:")
print(f"   Podium:  {PODIUM_X_COORDS[-1]:.0f}m x {Y_COORDS[-1]:.0f}m x {Z_COORDS[PODIUM_FLOORS-1]:.0f}m")
print(f"   Tower:   {TOWER_X_COORDS[-1]-TOWER_X_COORDS[0]:.0f}m x {Y_COORDS[-1]:.0f}m | Total: {Z_COORDS[-1]:.0f}m")

print(f"\nNODES: {n_nodes}")
print(f"   Podium: {n_podium} | Tower: {n_tower}")

print(f"\nELEMENTS:")
for etype, count in connectivity_df['element_type'].value_counts().items():
    print(f"   {etype:15s}: {count:5d}")
print(f"   {'TOTAL':15s}: {len(connectivity_df):5d}")

print(f"\nBRACE RATIO:")
print(f"   Front/Back (XZ): {n_xz}")
print(f"   Left/Right (YZ): {n_yz}")
print(f"   Ratio: {n_xz/max(n_yz,1):.1f} : 1")

BUILDING STRUCTURAL SUMMARY

GEOMETRY:
   Podium:  40m x 16m x 75m
   Tower:   24m x 16m | Total: 153m

NODES: 442
   Podium: 234 | Tower: 156

ELEMENTS:
   brace_floor    :   416
   column         :   372
   beam_x         :   312
   core_wall      :   300
   beam_y         :   260
   brace_xz       :   216
   brace_yz       :   144
   chevron        :   100
   brace_space    :    18
   TOTAL          :  2138

BRACE RATIO:
   Front/Back (XZ): 216
   Left/Right (YZ): 144
   Ratio: 1.5 : 1


In [149]:
# Statistics visualization
fig_stats = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Element Count', 'Length Distribution'],
    specs=[[{'type': 'pie'}, {'type': 'box'}]]
)

counts = connectivity_df['element_type'].value_counts()
fig_stats.add_trace(
    go.Pie(labels=[e.replace('_', ' ').title() for e in counts.index], values=counts.values, hole=0.4),
    row=1, col=1
)

for etype in connectivity_df['element_type'].unique():
    lengths = connectivity_df[connectivity_df['element_type'] == etype]['length']
    fig_stats.add_trace(
        go.Box(y=lengths, name=etype.replace('_', ' ').title()),
        row=1, col=2
    )

fig_stats.update_layout(title='Element Statistics', height=400, width=1000, showlegend=False)
fig_stats.show()

---
## 10. Export Data

In [150]:
import os

# Export to ../data/ folder
DATA_DIR = '../data'
os.makedirs(DATA_DIR, exist_ok=True)

position_df.to_csv(os.path.join(DATA_DIR, 'position_matrix.csv'), index=False)
connectivity_df.to_csv(os.path.join(DATA_DIR, 'connectivity_matrix.csv'), index=False)
np.savetxt(os.path.join(DATA_DIR, 'adjacency_matrix.csv'), adjacency_matrix, delimiter=',', fmt='%d')

np.savez(os.path.join(DATA_DIR, 'building_data.npz'),
         adjacency_matrix=adjacency_matrix,
         coords=coords,
         podium_x=PODIUM_X_COORDS,
         tower_x=TOWER_X_COORDS,
         y_coords=Y_COORDS,
         z_coords=Z_COORDS)

print(f"Exported to {os.path.abspath(DATA_DIR)}:")
print("  - position_matrix.csv")
print("  - connectivity_matrix.csv")
print("  - adjacency_matrix.csv")
print("  - building_data.npz")

Exported to /mnt/c/Users/lenovo/Desktop/DASK_NEW/data:
  - position_matrix.csv
  - connectivity_matrix.csv
  - adjacency_matrix.csv
  - building_data.npz


---
## 11. Stress Analysis Results

Load and visualize stress analysis results from OpenSeesPy analysis.

In [151]:
# Load stress analysis results
RESULTS_DIR = '../results/data'
stress_file = os.path.join(RESULTS_DIR, 'stress_results.csv')

if os.path.exists(stress_file):
    stress_df = pd.read_csv(stress_file)
    print(f"Loaded stress results: {len(stress_df)} elements")
    print(f"\nStress Statistics:")
    print(f"  Max combined stress: {stress_df['sigma_combined'].max():.3f} MPa")
    print(f"  Mean combined stress: {stress_df['sigma_combined'].mean():.3f} MPa")
    print(f"  Max utilization: {stress_df['utilization'].max():.2%}")
    print(f"\n  Elements > 50% utilization: {len(stress_df[stress_df['utilization'] > 0.5])}")
    print(f"  Elements > 80% utilization: {len(stress_df[stress_df['utilization'] > 0.8])}")
    print(f"  Overstressed (>100%): {len(stress_df[stress_df['utilization'] > 1.0])}")
    
    # Show top 10 critical elements
    print("\n  Top 10 Critical Elements:")
    critical = stress_df.nlargest(10, 'utilization')[['elem_id', 'orientation', 'sigma_combined', 'utilization', 'z_mid']]
    display(critical)
else:
    print(f"Stress results not found at {stress_file}")
    print("Run: python ../scripts/stress_analysis.py")

Loaded stress results: 2013 elements

Stress Statistics:
  Max combined stress: 34.121 MPa
  Mean combined stress: 0.418 MPa
  Max utilization: 284.34%

  Elements > 50% utilization: 36
  Elements > 80% utilization: 36
  Overstressed (>100%): 36

  Top 10 Critical Elements:


,elem_id,orientation,sigma_combined,utilization,z_mid
1403,1404,vertical,34.120917,2.843410,150000.0
1400,1401,vertical,34.120917,2.843410,150000.0
1401,1402,vertical,34.120917,2.274728,150000.0
1402,1403,vertical,34.120917,2.274728,150000.0
1398,1399,vertical,24.591763,2.049314,144000.0
1397,1398,vertical,24.591763,2.049314,144000.0
1385,1386,vertical,24.591577,2.049298,126000.0
1386,1387,vertical,24.591577,2.049298,126000.0
1361,1362,vertical,24.591577,2.049298,90000.0
1362,1363,vertical,24.591577,2.049298,90000.0


In [152]:
# 3D Stress Visualization
if 'stress_df' in dir() and len(stress_df) > 0:
    fig_stress = go.Figure()
    
    # Color elements by utilization
    for idx, row in stress_df.iterrows():
        util = row['utilization']
        if util > 1.0:
            color = 'red'
            width = 4
        elif util > 0.8:
            color = 'orange'
            width = 3
        elif util > 0.5:
            color = 'yellow'
            width = 2
        else:
            color = 'green'
            width = 1
        
        fig_stress.add_trace(go.Scatter3d(
            x=[row['x1'], row['x2']],
            y=[row['y1'], row['y2']],
            z=[row['z1'], row['z2']],
            mode='lines',
            line=dict(color=color, width=width),
            hovertemplate=(
                f"Element: {row['elem_id']}<br>"
                f"σ_combined: {row['sigma_combined']:.3f} MPa<br>"
                f"Utilization: {row['utilization']:.1%}<br>"
                f"<extra></extra>"
            ),
            showlegend=False
        ))
    
    # Add legend entries
    for label, color in [('Overstressed (>100%)', 'red'), ('High (80-100%)', 'orange'),
                         ('Medium (50-80%)', 'yellow'), ('OK (<50%)', 'green')]:
        fig_stress.add_trace(go.Scatter3d(
            x=[None], y=[None], z=[None],
            mode='lines',
            line=dict(color=color, width=3),
            name=label
        ))
    
    # Highlight critical elements
    critical_50 = stress_df[stress_df['utilization'] > 0.5]
    fig_stress.add_trace(go.Scatter3d(
        x=critical_50['x_mid'],
        y=critical_50['y_mid'],
        z=critical_50['z_mid'],
        mode='markers',
        marker=dict(size=5, color='red', symbol='diamond'),
        name=f'Critical ({len(critical_50)})',
    ))
    
    fig_stress.update_layout(
        title='3D Stress Visualization - Element Utilization',
        scene=dict(
            xaxis_title='X (mm)',
            yaxis_title='Y (mm)',
            zaxis_title='Z (mm)',
            aspectmode='data'
        ),
        legend=dict(x=0.02, y=0.98),
        width=1000, height=800
    )
    fig_stress.show()
else:
    print("Stress data not loaded. Run the stress analysis first.")

In [153]:
# Stress Distribution Charts
if 'stress_df' in dir() and len(stress_df) > 0:
    fig_stress_charts = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Stress vs Height',
            'Utilization Distribution',
            'Stress by Element Orientation',
            'Axial vs Bending Stress'
        )
    )
    
    # 1. Stress vs Height
    fig_stress_charts.add_trace(
        go.Scatter(
            x=stress_df['sigma_combined'],
            y=stress_df['z_mid'],
            mode='markers',
            marker=dict(
                size=4,
                color=stress_df['utilization'],
                colorscale='RdYlGn_r',
                showscale=True,
                colorbar=dict(title='Util', x=0.45, len=0.4, y=0.8)
            ),
            hovertemplate="σ: %{x:.2f} MPa<br>Z: %{y:.0f} mm<extra></extra>"
        ),
        row=1, col=1
    )
    
    # 2. Utilization Histogram
    fig_stress_charts.add_trace(
        go.Histogram(
            x=stress_df['utilization'] * 100,
            nbinsx=30,
            marker_color='steelblue'
        ),
        row=1, col=2
    )
    # Add threshold lines
    for thresh in [50, 80, 100]:
        fig_stress_charts.add_vline(x=thresh, line_dash="dash", 
                                     line_color="red" if thresh==100 else "orange",
                                     row=1, col=2)
    
    # 3. Stress by Orientation (box plot)
    for orient in stress_df['orientation'].unique():
        data = stress_df[stress_df['orientation'] == orient]['sigma_combined']
        fig_stress_charts.add_trace(
            go.Box(y=data, name=orient, boxpoints='outliers'),
            row=2, col=1
        )
    
    # 4. Axial vs Bending
    fig_stress_charts.add_trace(
        go.Scatter(
            x=stress_df['sigma_axial'],
            y=stress_df['sigma_bending'],
            mode='markers',
            marker=dict(
                size=4,
                color=stress_df['utilization'],
                colorscale='RdYlGn_r',
            ),
            hovertemplate="Axial: %{x:.2f}<br>Bending: %{y:.2f}<extra></extra>"
        ),
        row=2, col=2
    )
    
    fig_stress_charts.update_xaxes(title_text="Combined Stress (MPa)", row=1, col=1)
    fig_stress_charts.update_yaxes(title_text="Height Z (mm)", row=1, col=1)
    fig_stress_charts.update_xaxes(title_text="Utilization (%)", row=1, col=2)
    fig_stress_charts.update_yaxes(title_text="Count", row=1, col=2)
    fig_stress_charts.update_xaxes(title_text="Orientation", row=2, col=1)
    fig_stress_charts.update_yaxes(title_text="Stress (MPa)", row=2, col=1)
    fig_stress_charts.update_xaxes(title_text="Axial Stress (MPa)", row=2, col=2)
    fig_stress_charts.update_yaxes(title_text="Bending Stress (MPa)", row=2, col=2)
    
    fig_stress_charts.update_layout(
        title='Stress Analysis Summary',
        height=800, width=1100,
        showlegend=False
    )
    fig_stress_charts.show()
else:
    print("Stress data not loaded.")

---
## 12. Earthquake Analysis Results

Compare SAP2000 and OpenSeesPy earthquake analysis results.

In [154]:
# Load earthquake analysis results
eq_sap_file = os.path.join(RESULTS_DIR, 'earthquake_results.csv')
eq_ops_file = os.path.join(RESULTS_DIR, 'earthquake_results_opensees.csv')

eq_results = {}

if os.path.exists(eq_sap_file):
    eq_results['SAP2000'] = pd.read_csv(eq_sap_file)
    print("SAP2000 Results:")
    display(eq_results['SAP2000'])
else:
    print(f"SAP2000 results not found at {eq_sap_file}")

if os.path.exists(eq_ops_file):
    eq_results['OpenSees'] = pd.read_csv(eq_ops_file)
    print("\nOpenSeesPy Results:")
    display(eq_results['OpenSees'])
else:
    print(f"OpenSeesPy results not found at {eq_ops_file}")

SAP2000 Results:


,Analysis,Max_Ux_mm,Max_Uy_mm
0,RSX,1.847757e-12,0.000260
1,RSY,7.893390e-05,166.906412
2,TH_X,0.000000e+00,0.000000
3,TH_Y,0.000000e+00,0.000000
4,ENVELOPE,7.893390e-05,166.906412



OpenSeesPy Results:


,Analysis,Max_Ux_mm,Max_Uy_mm
0,RSX,0.000177,0.000000
1,RSY,0.000000,19733.167453
2,TH_X,0.687869,0.001260
3,TH_Y,0.692843,85.705074
4,ENVELOPE,0.692843,19733.167453


In [155]:
# Earthquake Results Comparison Chart
if len(eq_results) > 0:
    fig_eq = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Max Ux Displacement', 'Max Uy Displacement')
    )
    
    colors = {'SAP2000': 'steelblue', 'OpenSees': 'coral'}
    
    for software, df in eq_results.items():
        # Filter out ENVELOPE row for bar chart
        df_filtered = df[df['Analysis'] != 'ENVELOPE']
        
        fig_eq.add_trace(
            go.Bar(
                x=df_filtered['Analysis'],
                y=df_filtered['Max_Ux_mm'],
                name=f'{software} Ux',
                marker_color=colors.get(software, 'gray'),
                opacity=0.8
            ),
            row=1, col=1
        )
        
        fig_eq.add_trace(
            go.Bar(
                x=df_filtered['Analysis'],
                y=df_filtered['Max_Uy_mm'],
                name=f'{software} Uy',
                marker_color=colors.get(software, 'gray'),
                opacity=0.8
            ),
            row=1, col=2
        )
    
    fig_eq.update_xaxes(title_text="Analysis Type", row=1, col=1)
    fig_eq.update_xaxes(title_text="Analysis Type", row=1, col=2)
    fig_eq.update_yaxes(title_text="Displacement (mm)", row=1, col=1)
    fig_eq.update_yaxes(title_text="Displacement (mm)", row=1, col=2)
    
    fig_eq.update_layout(
        title='Earthquake Analysis Comparison: SAP2000 vs OpenSeesPy',
        barmode='group',
        height=500, width=1000,
        legend=dict(x=1.02, y=1)
    )
    fig_eq.show()
    
    # Summary comparison
    print("\n" + "="*60)
    print("EARTHQUAKE ANALYSIS SUMMARY")
    print("="*60)
    
    for software, df in eq_results.items():
        env = df[df['Analysis'] == 'ENVELOPE'].iloc[0]
        print(f"\n{software}:")
        print(f"  Max Ux: {env['Max_Ux_mm']:.2f} mm")
        print(f"  Max Uy: {env['Max_Uy_mm']:.2f} mm")
        
        # Calculate drift
        height_mm = 153000  # 153m in mm
        max_disp = max(env['Max_Ux_mm'], env['Max_Uy_mm'])
        drift = max_disp / height_mm * 100
        print(f"  Max Drift: {drift:.3f}%")
else:
    print("No earthquake results loaded.")


EARTHQUAKE ANALYSIS SUMMARY

SAP2000:
  Max Ux: 0.00 mm
  Max Uy: 166.91 mm
  Max Drift: 0.109%

OpenSees:
  Max Ux: 0.69 mm
  Max Uy: 19733.17 mm
  Max Drift: 12.897%


---
## 13. Summary & Next Steps

### Model Summary
- **Nodes**: 442 (234 podium + 156 tower + 52 chevron)
- **Elements**: 2162 total
- **Building Height**: 153m (26 floors)

### Analysis Workflow
1. **Generate Model**: Run this notebook to create position/connectivity matrices
2. **Export to SAP2000**: `python ../scripts/sap2000_export.py`
3. **Run Earthquake Analysis**: `python ../scripts/run_earthquake_analysis.py` (SAP2000)
4. **OpenSeesPy Analysis**: `python ../scripts/run_earthquake_opensees.py`
5. **Stress Analysis**: `python ../scripts/stress_analysis.py`

### Key Findings
- Structure is stiffer in X-direction than Y-direction
- Critical elements are vertical columns in upper floors
- Bending stress dominates over axial stress in critical elements

---
## 14. Material Quantity & Weight Analysis (DASK 2026)

Calculate total element lengths and weights based on DASK 2026 competition specifications:
- **Balsa Bars**: 250 pcs × 1000mm, 6mm × 6mm square section
- **Balsa Panels**: 35 pcs × 1000mm × 100mm × 3mm thick
- **Max Weight**: 1.0 kg (penalty up to 1.3 kg, disqualified above 1.3 kg)

In [156]:
from tabulate import tabulate

# ============================================
# DASK 2026 Material Specifications
# ============================================
# Frame elements: 6mm x 6mm square section
FRAME_WIDTH = 6.0   # mm
FRAME_AREA = FRAME_WIDTH ** 2  # 36 mm²

# Wall panels: 3mm thick
WALL_THICK = 3.0    # mm

# Available materials
AVAILABLE_BARS = 250  # pieces
BAR_LENGTH = 1000     # mm per bar
AVAILABLE_PANELS = 35  # pieces
PANEL_LENGTH = 1000   # mm
PANEL_WIDTH = 100     # mm

# Total available material
TOTAL_BAR_LENGTH = AVAILABLE_BARS * BAR_LENGTH  # 250,000 mm = 250 m
TOTAL_PANEL_AREA = AVAILABLE_PANELS * PANEL_LENGTH * PANEL_WIDTH  # 3,500,000 mm² = 3.5 m²

print("=" * 70)
print("DASK 2026 MATERIAL ANALYSIS")
print("=" * 70)
print(f"\nAvailable Materials:")
print(f"  Balsa Bars:   {AVAILABLE_BARS} pcs × {BAR_LENGTH}mm = {TOTAL_BAR_LENGTH/1000:.0f} m total")
print(f"  Balsa Panels: {AVAILABLE_PANELS} pcs × {PANEL_LENGTH}×{PANEL_WIDTH}mm = {TOTAL_PANEL_AREA/1e6:.2f} m² total")

# ============================================
# Calculate Element Lengths by Type
# ============================================
# Frame elements (use 6x6mm bars)
frame_types = ['beam_x', 'beam_y', 'column', 'brace_xz', 'brace_yz', 'chevron', 'brace_floor', 'brace_space']

# Core wall elements (use 3mm panels)
wall_types = ['core_wall']

length_summary = []

for etype in connectivity_df['element_type'].unique():
    elements = connectivity_df[connectivity_df['element_type'] == etype]
    total_length = elements['length'].sum()
    count = len(elements)
    avg_length = total_length / count if count > 0 else 0
    
    # Determine material type
    if etype in wall_types:
        material = 'Panel (3mm)'
        # For wall panels, calculate panel area needed
        # Wall height is the element length, width is the bay width (8m = 8000mm model scale)
        # But our model is in meters, so length is in meters
        # Each wall panel spans between nodes
    else:
        material = 'Bar (6×6mm)'
    
    length_summary.append({
        'Element Type': etype,
        'Count': count,
        'Total Length (m)': total_length,
        'Avg Length (m)': avg_length,
        'Material': material
    })

length_df = pd.DataFrame(length_summary)

print("\n" + "=" * 70)
print("ELEMENT LENGTH SUMMARY (Model Scale - meters)")
print("=" * 70)
print(tabulate(length_df, headers='keys', tablefmt='pretty', floatfmt='.2f', showindex=False))

# ============================================
# Calculate Total Lengths
# ============================================
# Frame elements total
frame_mask = length_df['Material'] == 'Bar (6×6mm)'
total_frame_length_m = length_df[frame_mask]['Total Length (m)'].sum()

# Wall elements total
wall_mask = length_df['Material'] == 'Panel (3mm)'
total_wall_length_m = length_df[wall_mask]['Total Length (m)'].sum()

print(f"\n{'='*70}")
print("TOTAL LENGTHS")
print(f"{'='*70}")
print(f"  Frame elements (6×6mm bars): {total_frame_length_m:.2f} m")
print(f"  Wall elements (3mm panels):  {total_wall_length_m:.2f} m")

DASK 2026 MATERIAL ANALYSIS

Available Materials:
  Balsa Bars:   250 pcs × 1000mm = 250 m total
  Balsa Panels: 35 pcs × 1000×100mm = 3.50 m² total

ELEMENT LENGTH SUMMARY (Model Scale - meters)
+--------------+-------+--------------------+--------------------+-------------+
| Element Type | Count |  Total Length (m)  |   Avg Length (m)   |  Material   |
+--------------+-------+--------------------+--------------------+-------------+
|    beam_x    |  312  |       2496.0       |        8.0         | Bar (6×6mm) |
|    beam_y    |  260  |       2080.0       |        8.0         | Bar (6×6mm) |
|    column    |  372  |       2286.0       | 6.145161290322581  | Bar (6×6mm) |
|   brace_xz   |  216  | 3177.0624000000007 | 14.708622222222225 | Bar (6×6mm) |
|   brace_yz   |  144  | 2118.0416000000005 | 14.708622222222225 | Bar (6×6mm) |
|   chevron    |  100  |      731.6612      |      7.316612      | Bar (6×6mm) |
| brace_space  |  18   |      237.9216      | 13.217866666666668 | Bar (6×6

In [157]:
# ============================================
# SCALE CONVERSION
# ============================================
# Model scale: 1:100 (model meters to maket mm)
# Model 1m = Maket 10mm (since floor height 6m model = 60mm maket)
SCALE_FACTOR = 1/100  # Model to real maket

# Convert model lengths (m) to maket lengths (mm)
# maket_mm = model_m * 1000 * SCALE_FACTOR = model_m * 10
MAKET_CONVERSION = 10  # model meters to maket mm

# Total lengths in MAKET scale (mm)
total_frame_length_mm = total_frame_length_m * MAKET_CONVERSION
total_wall_length_mm = total_wall_length_m * MAKET_CONVERSION

print(f"\n{'='*70}")
print("MAKET SCALE LENGTHS (1:100)")
print(f"{'='*70}")
print(f"  Frame elements: {total_frame_length_mm:.0f} mm = {total_frame_length_mm/1000:.2f} m")
print(f"  Wall elements:  {total_wall_length_mm:.0f} mm = {total_wall_length_mm/1000:.2f} m")

# ============================================
# VOLUME CALCULATION
# ============================================
# Frame volume: cross-section (36 mm²) × length (mm) = mm³
frame_volume_mm3 = FRAME_AREA * total_frame_length_mm

# Wall panels: For core walls, we need to calculate the panel AREA
# Core wall panels span: height (element length) × width (bay width 8m = 80mm in maket)
# The connectivity shows diagonal elements, but actual walls are rectangular panels

# Calculate core wall panel area
# Each wall panel has height = floor height and width = bay width
# From the core_wall elements, they connect corners of wall panels

# Let's calculate wall panel area more accurately
# Core walls span X=16-24 (8m) and Y=4-12 (8m) in model = 80mm × 80mm in maket
# Height per floor = 6m model = 60mm maket (except ground floor = 90mm)

# Number of core wall floors
core_wall_floors = TOTAL_FLOORS - 1  # 25 floor-to-floor connections

# Core has 6 wall faces per floor (front, back, left×2, right×2, center)
# From cell analysis: ~300 core_wall elements for 25 floors ≈ 12 diagonals per floor
# Each floor has: 2 front/back walls + 4 side walls = 6 wall panels

# Wall dimensions in maket scale:
wall_floor_height_mm = 60  # mm (typical floor, ground is 90mm)
wall_bay_width_mm = 80     # mm (8m model scale)

# Front/Back walls: 80mm wide × 60mm high each
# Side walls: 80mm wide × 60mm high each (but divided into 2 bays in Y)

# Approximate total wall panel area:
# We have ~6 wall panels per floor × 25 floors
# Average panel: 80mm × 60mm = 4800 mm² per panel
# But ground floor is taller: 90mm

n_wall_panels_estimate = 6 * (TOTAL_FLOORS - 1)  # 150 panels approximately

# More accurate: calculate from actual geometry
# Core spans X: 16-24 = 8m → 80mm
# Core spans Y: 0-16 = 16m → 160mm (but divided: 0-8, 8-16)
# Each floor has:
#   Front (Y=0): 80mm × height
#   Back (Y=16): 80mm × height  
#   Left (X=16): 80mm × height × 2 bays
#   Right (X=24): 80mm × height × 2 bays
#   Center (Y=8): 80mm × height (internal wall)

# Per typical floor: (80 + 80 + 80×2 + 80×2 + 80) × 60 = 560 × 60 = 33,600 mm²
# Ground floor: 560 × 90 = 50,400 mm²
# Total: 1 × 50,400 + 24 × 33,600 = 50,400 + 806,400 = 856,800 mm²

# Actually let's use the element data more directly
# Core wall "elements" are diagonals, 2 per wall panel face
# So 300 diagonals ≈ 150 wall panel faces

# Using simpler estimate based on diagonal lengths
# Each diagonal crosses a wall panel, wall area ≈ diagonal_length × panel_width / sqrt(2)
# Or more simply: wall_area = wall_height × wall_width

# Let's calculate properly from connectivity
core_wall_elements = connectivity_df[connectivity_df['element_type'] == 'core_wall']
core_wall_heights = []

for _, row in core_wall_elements.iterrows():
    n1, n2 = int(row['node_i']), int(row['node_j'])
    z1, z2 = coords[n1][2], coords[n2][2]
    height = abs(z2 - z1)  # in model meters
    core_wall_heights.append(height)

# Average wall height in model scale
avg_wall_height_m = np.mean(core_wall_heights) if core_wall_heights else 6.0

# Number of unique wall panels (2 diagonals per panel)
n_wall_panels = len(core_wall_elements) // 2

# Wall panel dimensions in maket
wall_height_mm = avg_wall_height_m * MAKET_CONVERSION  # model m to maket mm
wall_width_mm = 80  # 8m bay width in maket

# Total wall panel area
total_wall_area_mm2 = n_wall_panels * wall_height_mm * wall_width_mm

# Wall volume
wall_volume_mm3 = total_wall_area_mm2 * WALL_THICK

print(f"\n{'='*70}")
print("VOLUME CALCULATION")
print(f"{'='*70}")
print(f"\nFrame Elements:")
print(f"  Cross-section: {FRAME_AREA:.0f} mm² (6×6mm)")
print(f"  Total length:  {total_frame_length_mm:.0f} mm")
print(f"  Volume:        {frame_volume_mm3:,.0f} mm³ = {frame_volume_mm3/1e6:.4f} dm³")

print(f"\nWall Panels:")
print(f"  Number of panels: {n_wall_panels}")
print(f"  Avg panel size:   {wall_width_mm:.0f} × {wall_height_mm:.0f} mm")
print(f"  Panel thickness:  {WALL_THICK:.0f} mm")
print(f"  Total area:       {total_wall_area_mm2:,.0f} mm²")
print(f"  Volume:           {wall_volume_mm3:,.0f} mm³ = {wall_volume_mm3/1e6:.4f} dm³")

total_volume_mm3 = frame_volume_mm3 + wall_volume_mm3
print(f"\nTOTAL VOLUME: {total_volume_mm3:,.0f} mm³ = {total_volume_mm3/1e6:.4f} dm³")


MAKET SCALE LENGTHS (1:100)
  Frame elements: 178332 mm = 178.33 m
  Wall elements:  30245 mm = 30.24 m

VOLUME CALCULATION

Frame Elements:
  Cross-section: 36 mm² (6×6mm)
  Total length:  178332 mm
  Volume:        6,419,947 mm³ = 6.4199 dm³

Wall Panels:
  Number of panels: 150
  Avg panel size:   80 × 61 mm
  Panel thickness:  3 mm
  Total area:       734,400 mm²
  Volume:           2,203,200 mm³ = 2.2032 dm³

TOTAL VOLUME: 8,623,147 mm³ = 8.6231 dm³


In [158]:
# ============================================
# WEIGHT CALCULATION FOR DIFFERENT BALSA DENSITIES
# ============================================
# Balsa wood density range: 100-250 kg/m³ (typically 120-180 kg/m³)
# Convert to g/mm³: kg/m³ × 1e-6 = g/mm³

balsa_densities_kg_m3 = [100, 120, 140, 160, 180, 200, 220, 250]

weight_data = []

for rho in balsa_densities_kg_m3:
    rho_g_mm3 = rho * 1e-6  # Convert kg/m³ to g/mm³
    
    # Frame weight
    frame_weight_g = frame_volume_mm3 * rho_g_mm3
    
    # Wall weight
    wall_weight_g = wall_volume_mm3 * rho_g_mm3
    
    # Total weight
    total_weight_g = frame_weight_g + wall_weight_g
    total_weight_kg = total_weight_g / 1000
    
    # DASK compliance
    if total_weight_kg <= 1.0:
        status = "✓ OK"
        penalty = "None"
    elif total_weight_kg <= 1.3:
        status = "⚠ PENALTY"
        penalty = "Cost increase"
    else:
        status = "✗ DISQUALIFIED"
        penalty = "Not allowed"
    
    weight_data.append({
        'ρ (kg/m³)': rho,
        'Frame (g)': round(frame_weight_g, 1),
        'Wall (g)': round(wall_weight_g, 1),
        'Total (g)': round(total_weight_g, 1),
        'Total (kg)': round(total_weight_kg, 3),
        'DASK Status': status
    })

weight_df = pd.DataFrame(weight_data)

print(f"\n{'='*70}")
print("BUILDING WEIGHT vs BALSA DENSITY")
print(f"{'='*70}")
print(f"\nDASK 2026 Weight Limits:")
print(f"  ≤ 1.0 kg: OK (no penalty)")
print(f"  1.0-1.3 kg: Penalty (cost increase)")
print(f"  > 1.3 kg: DISQUALIFIED")
print()
print(tabulate(weight_df, headers='keys', tablefmt='pretty', showindex=False))

# ============================================
# MATERIAL USAGE CHECK
# ============================================
print(f"\n{'='*70}")
print("MATERIAL USAGE vs AVAILABLE")
print(f"{'='*70}")

# Frame bars usage
bars_needed = total_frame_length_mm / BAR_LENGTH
bars_usage_pct = (bars_needed / AVAILABLE_BARS) * 100

# Wall panels usage (area-based)
# Each panel is 1000mm × 100mm = 100,000 mm²
panel_area_each = PANEL_LENGTH * PANEL_WIDTH
panels_needed = total_wall_area_mm2 / panel_area_each
panels_usage_pct = (panels_needed / AVAILABLE_PANELS) * 100

print(f"\nFrame Bars (6×6mm):")
print(f"  Needed:    {bars_needed:.1f} bars ({total_frame_length_mm/1000:.1f} m)")
print(f"  Available: {AVAILABLE_BARS} bars ({TOTAL_BAR_LENGTH/1000:.0f} m)")
print(f"  Usage:     {bars_usage_pct:.1f}%")
if bars_needed > AVAILABLE_BARS:
    print(f"  ⚠ WARNING: Need {bars_needed - AVAILABLE_BARS:.1f} more bars!")
else:
    print(f"  ✓ OK: {AVAILABLE_BARS - bars_needed:.1f} bars remaining")

print(f"\nWall Panels (3mm thick):")
print(f"  Needed:    {panels_needed:.1f} panels ({total_wall_area_mm2/1e6:.3f} m²)")
print(f"  Available: {AVAILABLE_PANELS} panels ({TOTAL_PANEL_AREA/1e6:.2f} m²)")
print(f"  Usage:     {panels_usage_pct:.1f}%")
if panels_needed > AVAILABLE_PANELS:
    print(f"  ⚠ WARNING: Need {panels_needed - AVAILABLE_PANELS:.1f} more panels!")
else:
    print(f"  ✓ OK: {AVAILABLE_PANELS - panels_needed:.1f} panels remaining")


BUILDING WEIGHT vs BALSA DENSITY

DASK 2026 Weight Limits:
  ≤ 1.0 kg: OK (no penalty)
  1.0-1.3 kg: Penalty (cost increase)
  > 1.3 kg: DISQUALIFIED

+-----------+-----------+----------+-----------+------------+----------------+
| ρ (kg/m³) | Frame (g) | Wall (g) | Total (g) | Total (kg) |  DASK Status   |
+-----------+-----------+----------+-----------+------------+----------------+
|    100    |   642.0   |  220.3   |   862.3   |   0.862    |      ✓ OK      |
|    120    |   770.4   |  264.4   |  1034.8   |   1.035    |   ⚠ PENALTY    |
|    140    |   898.8   |  308.4   |  1207.2   |   1.207    |   ⚠ PENALTY    |
|    160    |  1027.2   |  352.5   |  1379.7   |    1.38    | ✗ DISQUALIFIED |
|    180    |  1155.6   |  396.6   |  1552.2   |   1.552    | ✗ DISQUALIFIED |
|    200    |  1284.0   |  440.6   |  1724.6   |   1.725    | ✗ DISQUALIFIED |
|    220    |  1412.4   |  484.7   |  1897.1   |   1.897    | ✗ DISQUALIFIED |
|    250    |  1605.0   |  550.8   |  2155.8   |   2.156  

In [159]:
# ============================================
# WEIGHT vs DENSITY VISUALIZATION
# ============================================
fig_weight = go.Figure()

# Total weight line
fig_weight.add_trace(go.Scatter(
    x=weight_df['ρ (kg/m³)'],
    y=weight_df['Total (kg)'],
    mode='lines+markers',
    name='Total Weight',
    line=dict(color='blue', width=3),
    marker=dict(size=10)
))

# Frame weight
fig_weight.add_trace(go.Scatter(
    x=weight_df['ρ (kg/m³)'],
    y=weight_df['Frame (g)'] / 1000,
    mode='lines+markers',
    name='Frame Weight',
    line=dict(color='green', width=2, dash='dash'),
    marker=dict(size=6)
))

# Wall weight
fig_weight.add_trace(go.Scatter(
    x=weight_df['ρ (kg/m³)'],
    y=weight_df['Wall (g)'] / 1000,
    mode='lines+markers',
    name='Wall Weight',
    line=dict(color='brown', width=2, dash='dash'),
    marker=dict(size=6)
))

# DASK limits
fig_weight.add_hline(y=1.0, line_dash="solid", line_color="orange", 
                      annotation_text="1.0 kg (OK limit)", annotation_position="right")
fig_weight.add_hline(y=1.3, line_dash="solid", line_color="red",
                      annotation_text="1.3 kg (DISQUALIFIED)", annotation_position="right")

# Shaded regions
fig_weight.add_hrect(y0=0, y1=1.0, fillcolor="green", opacity=0.1, line_width=0)
fig_weight.add_hrect(y0=1.0, y1=1.3, fillcolor="orange", opacity=0.1, line_width=0)
fig_weight.add_hrect(y0=1.3, y1=2.0, fillcolor="red", opacity=0.1, line_width=0)

fig_weight.update_layout(
    title='Building Weight vs Balsa Density (DASK 2026)',
    xaxis_title='Balsa Density ρ (kg/m³)',
    yaxis_title='Weight (kg)',
    yaxis=dict(range=[0, 2.0]),
    legend=dict(x=0.02, y=0.98),
    width=900, height=500
)
fig_weight.show()

# Summary table
print(f"\n{'='*70}")
print("SUMMARY: RECOMMENDED DENSITY RANGE")
print(f"{'='*70}")

# Find max density that stays under 1.0 kg
ok_densities = weight_df[weight_df['Total (kg)'] <= 1.0]['ρ (kg/m³)']
penalty_densities = weight_df[(weight_df['Total (kg)'] > 1.0) & (weight_df['Total (kg)'] <= 1.3)]['ρ (kg/m³)']

if len(ok_densities) > 0:
    max_ok_density = ok_densities.max()
    print(f"\n  ✓ Use balsa with ρ ≤ {max_ok_density} kg/m³ to stay under 1.0 kg")
else:
    print(f"\n  ⚠ Current design exceeds 1.0 kg even with lightest balsa!")
    
if len(penalty_densities) > 0:
    print(f"  ⚠ ρ = {penalty_densities.min()}-{penalty_densities.max()} kg/m³ → penalty zone (1.0-1.3 kg)")

# Typical balsa density
typical_rho = 160
typical_row = weight_df[weight_df['ρ (kg/m³)'] == typical_rho].iloc[0]
print(f"\n  Typical balsa (ρ={typical_rho} kg/m³): {typical_row['Total (kg)']:.3f} kg → {typical_row['DASK Status']}")


SUMMARY: RECOMMENDED DENSITY RANGE

  ✓ Use balsa with ρ ≤ 100 kg/m³ to stay under 1.0 kg
  ⚠ ρ = 120-140 kg/m³ → penalty zone (1.0-1.3 kg)

  Typical balsa (ρ=160 kg/m³): 1.380 kg → ✗ DISQUALIFIED
